In [ ]:
import requests
import pandas as pd
import io
from pathlib import Path

In [ ]:
# Help with querying API: https://dev.socrata.com/docs/queries/

fileName = Path('dohmh_export.csv')
if (Path.exists(fileName)):
    df = pd.read_csv(fileName)
else:
    url = 'https://data.cityofnewyork.us/resource/43nn-pn8j.csv'
    params = {
        '$limit': 500000
    }
    response = requests.get(url, params)
    csv = response.content.decode('utf-8')
    data = io.StringIO(csv)
    df = pd.read_csv(data)
    df.to_csv(fileName, header = True, index = False)

In [4]:
df

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,...,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,location_point1
0,50163690,PARIS BAGUETTE,Manhattan,990,AVENUE OF THE AMERICAS,10018.0,5169571119,NaN,1900-01-01T00:00:00.000,NaN,...,NaN,40.751404,-73.986599,105.0,4.0,8400.0,1015919.0,1.008380e+09,MN17,NaN
1,50165178,Richmond Pizza Hut,Queens,11222,101ST AVE,11419.0,3475536506,NaN,1900-01-01T00:00:00.000,NaN,...,NaN,40.688351,-73.831586,409.0,29.0,10800.0,4539299.0,4.094300e+09,QN54,NaN
2,41236580,DUNKIN,Queens,10005,QUEENS BOULEVARD,11375.0,7187933690,Donuts,2024-09-09T00:00:00.000,Establishment re-opened by DOHMH.,...,Cycle Inspection / Reopening Inspection,40.726899,-73.853544,406.0,29.0,71303.0,4050969.0,4.021190e+09,QN17,NaN
3,50163819,SEOUL SWEETS INC,Manhattan,308,5 AVENUE,10001.0,6467070822,NaN,1900-01-01T00:00:00.000,NaN,...,NaN,40.746817,-73.985752,105.0,4.0,7600.0,1015818.0,1.008330e+09,MN17,NaN
4,50123040,MIGARBA WINE BAR,Manhattan,310,BLEECKER STREET,10014.0,9175888361,NaN,1900-01-01T00:00:00.000,NaN,...,NaN,40.732740,-74.003886,102.0,3.0,7300.0,1010017.0,1.005880e+09,MN23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274701,50130416,EMMY SQUARED PIZZA,Brooklyn,315,5 AVENUE,11215.0,8455518605,Pizza,2024-01-18T00:00:00.000,Violations were cited in the following area(s).,...,Pre-permit (Operational) / Initial Inspection,40.672922,-73.983161,306.0,39.0,13500.0,3020923.0,3.009750e+09,BK37,NaN
274702,41375676,AU ZA'ATAR,Manhattan,188,AVENUE A,10009.0,2122545660,Middle Eastern,2022-06-14T00:00:00.000,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.728923,-73.981426,103.0,2.0,3400.0,1005166.0,1.004050e+09,MN22,NaN
274703,40996589,GENA'S GRILL,Manhattan,210,1 AVENUE,10009.0,2124733700,Latin American,2025-03-13T00:00:00.000,Violations were cited in the following area(s).,...,Cycle Inspection / Compliance Inspection,40.730395,-73.983240,103.0,2.0,3400.0,1005981.0,1.004400e+09,MN22,NaN
274704,50095004,251 GINZA SUSHI,Manhattan,251,EAST 35 STREET,10016.0,2128893333,Japanese,2022-06-14T00:00:00.000,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.745786,-73.976199,106.0,4.0,7800.0,1020188.0,1.009160e+09,MN20,NaN


In [5]:
df['cuisine_description'].unique()

array([nan, 'Donuts', 'Coffee/Tea', 'African', 'Barbecue',
       'Pancakes/Waffles', 'Sandwiches', 'Chinese', 'Southeast Asian',
       'French', 'Latin American', 'Tex-Mex', 'Mexican', 'American',
       'Korean', 'Japanese', 'Hamburgers', 'Spanish', 'Greek',
       'Bakery Products/Desserts', 'Caribbean', 'Indian',
       'Eastern European', 'Pizza', 'Italian', 'Other', 'Mediterranean',
       'Thai', 'Bangladeshi', 'Portuguese', 'Middle Eastern', 'Salads',
       'Asian/Asian Fusion', 'Chicken', 'Turkish', 'Bagels/Pretzels',
       'Brazilian', 'German', 'Peruvian',
       'Sandwiches/Salads/Mixed Buffet', 'Vegetarian', 'Seafood', 'Irish',
       'Russian', 'Jewish/Kosher', 'Frozen Desserts', 'Bottled Beverages',
       'Creole', 'Juice, Smoothies, Fruit Salads', 'Tapas', 'Filipino',
       'Pakistani', 'Chilean', 'Steakhouse', 'Vegan', 'Hotdogs/Pretzels',
       'Egyptian', 'Ethiopian', 'Moroccan', 'Australian', 'Scandinavian',
       'New American', 'Chinese/Japanese', 'Continent

In [12]:
df = df[df['cuisine_description'].notna()].copy()